# Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов

### 2.1. Подготовка данных

**Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче нашего соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы (хотя можете проверить), продемонстрируем всю резвость VW в задаче классификации на 400 классов. Исходные данные все те же самые, но выделено 400 пользователей, и решается задача их идентификации. Скачайте данные отсюда – файлы train_sessions_400users.csv и test_sessions_400users.csv.**

In [1]:
import os
import pandas as pd
import numpy as np
import scipy.sparse as sps
from time import time
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import preprocessing

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'capstone_user_identification'

**Загрузим обучающую и тестовую выборки. Можете заметить, что тестовые сессии здесь по времени четко отделены от сессий в обучающей выборке.**

In [3]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), 
                           index_col='session_id')

In [4]:
test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), 
                           index_col='session_id')

In [5]:
train_df_400.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273


**Видим, что в обучающей выборке 182793 сессий, в тестовой – 46473, и сессии действительно принадлежат 400 различным пользователям.**

In [6]:
train_df_400.shape, test_df_400.shape, train_df_400['user_id'].nunique()

((182793, 21), (46473, 20), 400)

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить LabelEncoder, да еще и +1 потом добавить (LabelEncoder переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.**

In [7]:
y = train_df_400.user_id
class_encoder = preprocessing.LabelEncoder()
y_for_vw = class_encoder.fit_transform(y)+1

**Далее будем сравнивать VW с SGDClassifier и с логистической регрессией. Всем моделям этим нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:**

* объедините обучающиую и тестовую выборки
* выберите только сайты (признаки от 'site1' до 'site10')
* замените пропуски на нули (сайты у нас нумеровались с 0)
* переведите в разреженный формат csr_matrix
* разбейте обратно на обучающую и тестовую части

In [8]:
sites = ['site' + str(i) for i in range(1, 11)]

In [9]:
train_test_df = pd.concat([train_df_400, test_df_400])

/home/zmey56/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [10]:
train_test_df_sites = train_test_df[sites]

In [11]:
train_test_df_sites.isnull().sum().sum()

81858

In [12]:
train_test_df_sites = train_test_df_sites.fillna(0)

In [13]:
train_test_df_sites.isnull().sum().sum()

0

In [14]:
train_test_df_sites = train_test_df_sites.astype(int)

In [15]:
train_test_df_sites.shape

(229266, 10)

In [16]:
tmp = test_df_400[sites].fillna(0).astype(int)

In [17]:
# 25082019

def create_sparse_matrix(dataframe):
    tmp_arr = np.array(dataframe)
    row = 0
    rows = []
    cols = []
    data = []

    for arr in tmp_arr:
        #print(arr)
        for i, val in enumerate(arr):
            if val != 0:                
                data.append(val)
                cols.append(i)
                rows.append(row)
        row = row + 1
        
    return(sps.coo_matrix((data, (rows, cols))))

In [18]:
t_start = time()

idx_split = train_df_400.shape[0]

#tmp_sparse = csr_matrix(create_sparse_matrix(tmp))

train_test_sparse = csr_matrix(create_sparse_matrix(train_test_df_sites))

X_train_sparse = train_test_sparse[:idx_split, :]
X_test_sparse = train_test_sparse[idx_split:,:]

#y = train_df.target

print('Train DF size: {0}\nTest DF size: {1}\nTarget size: {2}'.format(
    str(X_train_sparse.shape), str(X_test_sparse.shape), str(y.shape)))

print("Time elapse: ", time() - t_start)

Train DF size: (182793, 10)
Test DF size: (46473, 10)
Target size: (182793,)
Time elapse:  1.8635473251342773


### 2.2. Валидация по отложенной выборке

**Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.**

In [19]:
train_share = int(.7 * train_df_400.shape[0])
train_df_part = train_df_400[sites].iloc[:train_share, :]
valid_df = train_df_400[sites].iloc[train_share:, :]
X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [20]:
y_train_part = y[:train_share]
y_valid = y[train_share:]
y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]

Реализуйте функцию, arrays_to_vw, переводящую обучающую выборку в формат Vowpal Wabbit.

Вход:

* X – матрица NumPy (обучающая выборка)
* y (необяз.) - вектор ответов (NumPy). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
* train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
* out_file – путь к файлу .vw, в который будет произведена запись

Детали:

* надо пройтись по всем строкам матрицы X и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора y и знак-разделитель |
* в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [84]:
def arrays_to_vw(X, y=None, train=True, out_file='tmp.vw'):
    X = np.nan_to_num(X)
    X = X.astype(int)
    
    with open(out_file, 'w') as f:
        print(X.shape)
        for i in range(X.shape[0]):
            string =  ' '.join([str(x) for x in X[i]])
            if y is None:
                #print("NONE")
                f.write(str(1) + " | " + string + "\n")
            else:
                f.write(str(y[i]) + " | " + string + "\n")

**Примените написанную функцию к части обучащей выборки (train_df_part, y_train_part_for_vw), к отложенной выборке (valid_df, y_valid_for_vw), ко всей обучающей выборке и ко всей тестовой выборке. Обратите внимание, что на вход наш метод принимает именно матрицы и вектора NumPy.**

In [85]:
t_start = time()
#tmp = create_sparse_matrix(train_df_part)


arrays_to_vw(train_df_part.values, y_train_part_for_vw, True, 'kaggle_data/train_part.vw')
arrays_to_vw(valid_df.values, y_valid_for_vw, False, 'kaggle_data/valid.vw')
arrays_to_vw(train_df_400[sites].values, y_for_vw, True, 'kaggle_data/train.vw')
arrays_to_vw(test_df_400[sites].values, None, False, 'kaggle_data/test.vw')

print("Time elapse: ", time() - t_start)

(127955, 10)
(54838, 10)
(182793, 10)
(46473, 10)
Time elapse:  3.370938301086426


**Обучите модель Vowpal Wabbitна выборке train_part.vw. Укажите, что решается задача классификации с 400 классами (--oaa), сделайте 3 прохода по выборке (--passes). Задайте некоторый кэш-файл (--cache_file, можно просто указать флаг -c), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента -k). Также укажите значение параметра b=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Наконец, укажите random_seed=17. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.**

In [86]:
train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [87]:
!head -3 kaggle_data/test.vw

1 | 9 304 308 307 91 308 312 300 305 309
1 | 838 504 68 11 838 11 838 886 27 305
1 | 190 192 8 189 191 189 190 2375 192 8


In [88]:
!head -3 kaggle_data/train_part.vw

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


In [89]:
!head -3 kaggle_data/valid.vw

4 | 7 923 923 923 11 924 7 924 838 7
160 | 91 198 11 11 302 91 668 311 310 91
312 | 27085 848 118 118 118 118 11 118 118 118


In [90]:
#!vw --oaa 400 --passes 3 --cache_file train.cache -b 26 --random_seed 17 -k -d $1 -f kaggle_data/vw.model

In [91]:
#!vw --oaa 400 kaggle_data/train_part.vw -f kaggle_data/oaa.model --loss_function=hinge

In [102]:
!vw --oaa 400 --passes 3 -c -k kaggle_data/train_part.vw -b 26 -f kaggle_data/oaa.model --random_seed 17

final_regressor = kaggle_data/oaa.model
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = kaggle_data/train_part.vw.cache
Reading datafile = kaggle_data/train_part.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976562 0.984375          128          128.0      348      346       11
0.941406 0.906250     

**Запишите прогнозы на выборке valid.vw в vw_valid_pred.csv.**

In [103]:
!vw -i kaggle_data/oaa.model -t -d kaggle_data/valid.vw -p kaggle_data/vw_valid_pred.csv --quiet

**Считайте прогнозы kaggle_data/vw_valid_pred.csv из файла и посмотрите на долю правильных ответов на отложенной части.**

In [104]:
y_pred = pd.read_csv('kaggle_data/vw_valid_pred.csv', header=None)

In [106]:
y_valid

session_id
127956      7
127957    402
127958    798
127959    360
127960    815
127961    370
127962    328
127963    616
127964     59
127965    558
127966    534
127967    100
127968    120
127969    812
127970    778
127971    877
127972    364
127973    832
127974    613
127975    495
127976    613
127977    475
127978    917
127979    179
127980    107
127981    586
127982     30
127983    832
127984    432
127985    753
         ... 
182764    120
182765    361
182766    318
182767    310
182768    205
182769    789
182770    200
182771    803
182772    939
182773    292
182774      7
182775    935
182776    829
182777    208
182778    725
182779    954
182780    829
182781    575
182782    778
182783    572
182784     48
182785    393
182786    371
182787    862
182788    512
182789    183
182790    448
182791    632
182792    232
182793    635
Name: user_id, Length: 54838, dtype: int64

In [82]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix

In [83]:
accuracy_score(y_pred, y_valid)

0.0

In [65]:
train_part.shape

NameError: name 'train_part' is not defined

**ну не получается по схеме. попробую через другую разряженную матрицу**

In [33]:
def create_sparse_matrix(dataframe):
    tmp_arr = np.array(dataframe)
    row = 0
    rows = []
    cols = []
    data = []

    for arr in tmp_arr:
        unique, counts = np.unique(arr, return_counts=True)
        #print(dict(zip(unique, counts)))
        for key, value in dict(zip(unique, counts)).items():
            if key != 0:
                rows.append(row)
                cols.append(key-1)
                data.append(value)
        row = row + 1
        
    return(sps.coo_matrix((data, (rows, cols))))

In [34]:
t_start = time()

idx_split = train_df_400.shape[0]

#tmp_sparse = csr_matrix(create_sparse_matrix(tmp))

train_test_sparse = csr_matrix(create_sparse_matrix(train_test_df_sites))

X_train_sparse = train_test_sparse[:idx_split, :]
X_test_sparse = train_test_sparse[idx_split:,:]

#y = train_df.target

print('Train DF size: {0}\nTest DF size: {1}\nTarget size: {2}'.format(
    str(X_train_sparse.shape), str(X_test_sparse.shape), str(y.shape)))

print("Time elapse: ", time() - t_start)

Train DF size: (182793, 36656)
Test DF size: (46473, 36656)
Target size: (182793,)
Time elapse:  6.0806169509887695


In [35]:
train_df_part = X_train_sparse[:train_share, :]
valid_df = X_train_sparse[train_share:, :]

In [36]:
tmp = [((i, j), train_df_part[i,j]) for i, j in zip(*train_df_part.nonzero())]

In [47]:
tmp

[((0, 23712), array(8)),
 ((0, 23719), array(2)),
 ((1, 44), array(2)),
 ((1, 664), array(1)),
 ((1, 5319), array(3)),
 ((1, 8724), array(2)),
 ((1, 8725), array(1)),
 ((1, 8726), array(1)),
 ((2, 18), array(1)),
 ((2, 302), array(8)),
 ((2, 308), array(1)),
 ((3, 924), array(1)),
 ((3, 1239), array(1)),
 ((3, 1343), array(2)),
 ((3, 1344), array(2)),
 ((3, 1345), array(1)),
 ((3, 1358), array(2)),
 ((3, 1359), array(1)),
 ((4, 9), array(1)),
 ((4, 10), array(3)),
 ((4, 51), array(2)),
 ((4, 84), array(4)),
 ((5, 82), array(2)),
 ((5, 84), array(1)),
 ((5, 924), array(1)),
 ((5, 1239), array(1)),
 ((5, 1343), array(1)),
 ((5, 1344), array(1)),
 ((5, 1345), array(1)),
 ((5, 1358), array(2)),
 ((6, 13584), array(9)),
 ((6, 13585), array(1)),
 ((7, 29310), array(1)),
 ((7, 29311), array(3)),
 ((8, 6), array(1)),
 ((8, 9), array(1)),
 ((8, 17), array(1)),
 ((8, 20), array(1)),
 ((8, 21), array(1)),
 ((8, 61), array(1)),
 ((8, 71), array(1)),
 ((8, 1383), array(1)),
 ((8, 1385), array(1)),


In [50]:
print(tmp[0][0][1])
print(tmp[0][1])
print(tmp[1][0][1])

23712
8
23719


In [100]:
y_true = y_valid_for_vw.reshape(y_valid_for_vw.shape[0], 1)

In [101]:
y_true

array([[  4],
       [160],
       [312],
       ...,
       [254],
       [ 91],
       [256]])

In [58]:
tmp.shape

(46473, 10)

In [57]:
y_for_vw.shape

(182793,)

In [77]:
c.row

array([     0,      0,      1, ..., 229265, 229265, 229265], dtype=int32)

In [75]:
len(c.col)

1294146

In [87]:
for index, value in zip(colum, data):
    if index not in val_dict:
        val_dict[index] = value
    else:
        val_dict[index] += value

In [95]:
train_test_sparse[0].shape

(1, 36656)

In [100]:
train_test_sparse.item()

AttributeError: item not found

In [41]:
#y_train_part 
#y_valid 
#y_train_part_for_vw 
y_valid_for_vw

array([  4, 160, 312, ..., 254,  91, 256])